# Spaces

Spaces are handled in `ebcc` according to the following nomenclature:

                ─┬─ ┌──────────┐
                 │  │  frozen  │
                 │  ├──────────┤ ─┬─
         virtual │  │  active  │  │
                 │  ├──────────┤  │ correlated
                 │  │ inactive │  │
                ─┼─ ├══════════┤ ─┼─
                 │  │ inactive │  │
                 │  ├──────────┤  │ correlated
        occupied │  │  active  │  │
                 │  ├──────────┤ ─┴─
                 │  │  frozen  │
                ─┴─ └──────────┘

A plain coupled cluster calculation will operate within the correlated space. The simplest use of this system is frozen core calculations.

First, we again find system and mean-field using PySCF, and initialise an appropriate logger.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="cc-pvdz", verbose=0)
mf = scf.RHF(mol).run()

In [2]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

The `Space` class is constructed by providing boolean arrays indicating whether an orbital at that index is occupied, frozen, or active. In this example we freeze the two lowest energy MOs.

In [3]:
from ebcc import Space

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True

space = Space(occupied, frozen, active)

print("Space:", space)

Space: (7o, 21v) [(2o, 0v) frozen]


The `active` space is used for methods that differentiate between the correlated orbitals in order to treat a subset of orbitals at a higher level of theory. The `frozen` and `active` arrays must be disjoint.

In [4]:
occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:1] = True
active[np.sum(mf.mo_occ > 0) - 1] = True
space = Space(occupied, frozen, active)
print("Space:", space)

try:
    active = frozen
    space = Space(occupied, frozen, active)
except ValueError as e:
    print("Error:", e)

Space: (7o, 21v) [(1o, 0v) frozen, (1o, 0v) active]
Error: Frozen and active orbitals must be mutually exclusive.


The space can be used in a calculation to perform the frozen-core CC calculation. All methods should be compatible with this procedure, as the code generation is agnostic to the definition of the space.

In [5]:
from ebcc import REBCC

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True
space = Space(occupied, frozen, active)

cc2 = REBCC(mf, space=space, log=log)
cc2.kernel()

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.4
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 21v) [(2o, 0v) frozen]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3069910359    -109.2607872768
   1    -0.2961558041    -109.2499520450     1.084e-02     1.453e-02
   2    -0.3083989049    -109.2621951458     1.224e-02     7.635e-03
   3    -0.3082114791    -109.2620077200     1.874e-04     2.606e-03
   4    -0.3098129983    -109.2636092392     1.602e-03     2.384e-03
   5    -0.3097737022    -109.2635699430     3.930e-05     4.774e-04
   6    -0.3097803152 

-0.30978110088277533

An example of the use of the so-called active space is CCSDt', where the third-order amplitudes span only the active orbitals. We can mix this with frozen orbitals to make use of all three tiers of the space. In this example we freeze the two lowest energy orbitals, and allow the third-order amplitudes to span the two highest energy occpued and two lowest energy virtual MOs.

In [6]:
occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
active[np.sum(mf.mo_occ > 0) - 1] = True
active[np.sum(mf.mo_occ > 0) - 2] = True
active[np.sum(mf.mo_occ > 0)    ] = True
active[np.sum(mf.mo_occ > 0) + 1] = True
frozen[:2] = True
space = Space(occupied, frozen, active)

ccsdt = REBCC(mf, ansatz="CCSDt'", space=space, log=log)
ccsdt.kernel()


RCCSDt'
*******

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSDt'

Space: (7o, 21v) [(2o, 0v) frozen, (2o, 2v) active]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3069910359    -109.2607872768
   1    -0.2961558041    -109.2499520450     1.084e-02     1.453e-02
   2    -0.3083989049    -109.2621951458     1.224e-02     7.635e-03
   3    -0.3082114791    -109.2620077200     1.874e-04     2.606e-03
   4    -0.3098129983    -109.2636092392     1.602e-03     2.384e-03
   5    -0.3097737022    -109.2635699430     3.930e-05     4.774e-04
   6    -0.3097803152    -109.2635765560     6.613e-06     4.070e-05
   7    -0.3097817651    -109.2635780059     1.450e-06     5.862e-06
   8    -0.3097810385    -109.2635772794     7.266e-07     1.700e-06
   9    -0.3097810880    -109.2635773289     4.950e-08     2.441e-07
  10    -0.3097811033    -1

-0.309781100882775

Frozen natural orbital (FNO) calculations can be easily performed using the helper function to construct the space.

In [7]:
from ebcc.ham.space import construct_fno_space

no_coeff, no_occ, no_space = construct_fno_space(mf, occ_tol=1e-3)

fno_ccsd = REBCC(mf, mo_coeff=no_coeff, mo_occ=no_occ, space=no_space, log=log)
fno_ccsd.kernel()


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 21v) [(0o, 4v) frozen]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3025875079    -109.2563837488
   1    -0.2914209050    -109.2452171459     1.117e-02     1.382e-02
   2    -0.3030681327    -109.2568643736     1.165e-02     6.529e-03
   3    -0.3029959077    -109.2567921486     7.222e-05     2.033e-03
   4    -0.3044176481    -109.2582138890     1.422e-03     1.723e-03
   5    -0.3043968157    -109.2581930566     2.083e-05     3.246e-04
   6    -0.3044029136    -109.2581991545     6.098e-06     2.812e-05
   7    -0.3044041681    -109.2582004090     1.254e-06     4.942e-06
   8    -0.3044036497    -109.2581998906     5.183e-07     1.392e-06
   9    -0.3044036658    -109.2581999067     1.602e-08     1.327e-07
  10    -0.3044036764    -109.2581999173     1.067

-0.30440367492534665

External corrections are also supported, and use the designated active space to signify the orbitals in which the external amplitudes are calculated.

In [8]:
from pyscf import fci, ao2mo
from ebcc.ext.fci import extract_amplitudes_restricted

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
active[np.sum(mf.mo_occ > 0) - 4 : np.sum(mf.mo_occ > 0) + 4] = True
space = Space(occupied, frozen, active)

mo = mf.mo_coeff[:, space.active]
h1e = np.einsum("pq,pi,qj->ij", mf.get_hcore(), mo, mo)
h2e = ao2mo.kernel(mf._eri, mo, compact=False).reshape((mo.shape[-1],) * 4)
ci = fci.direct_spin1.FCI()
ci.kernel(h1e, h2e, space.nact, space.naocc * 2)
amplitudes = extract_amplitudes_restricted(ci, space)

eccc = REBCC(mf, ansatz="CCSD", space=space, log=log)
eccc.external_correction(amplitudes, mixed_term_strategy="update")


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 21v) [(4o, 4v) active]

Applying external corrections.
 > mixed_terms_strategy:  fixed

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3112847700    -109.2650810109
   1    -0.3003018073    -109.2540980482     1.098e-02     1.452e-02
   2    -0.3125606000    -109.2663568409     1.226e-02     7.573e-03
   3    -0.3123392822    -109.2661355231     2.213e-04     2.547e-03
   4    -0.3139377696    -109.2677340105     1.598e-03     2.350e-03
   5    -0.3139042579    -109.2677004988     3.351e-05     4.794e-04
   6    -0.3139103941    -109.2677066350     6.136e-06     3.914e-05
   7    -0.3139119247    -109.2677081656     1.531e-06     5.661e-06
   8    -0.3139111792    -109.2677074201     7.455e-07     1.678e-06
   9    -0.3139112246    -109.2677074655     4.540e-08 

-0.31391123618855044

Alternatively, one can use tailoring with external amplitudes.

In [9]:
eccc = REBCC(mf, ansatz="CCSD", space=space, log=log)
eccc.tailor(amplitudes)


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 21v) [(4o, 4v) active]

Applying tailoring.

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3112847700    -109.2650810109
   1    -0.2670966623    -109.2208929032     4.419e-02     3.630e-01
   2    -0.2869167900    -109.2407130309     1.982e-02     1.971e-02
   3    -0.2845173180    -109.2383135589     2.399e-03     3.134e-03
   4    -0.2856479762    -109.2394442171     1.131e-03     9.649e-04
   5    -0.2856167978    -109.2394130387     3.118e-05     2.876e-04
   6    -0.2856247272    -109.2394209680     7.929e-06     4.926e-05
   7    -0.2856256760    -109.2394219169     9.488e-07     3.685e-06
   8    -0.2856258088    -109.2394220497     1.328e-07     9.818e-07
   9    -0.2856258924    -109.2394221333     8.360e-08     2.369e-07
  10    -0.2856258985    -109

-0.2856259026309606